In [ ]:
# from main_library import *

# # ################################
# # ## data load / adjustements*
# # extractDate = date_load()

# # ### pour l'instant ne fonctionne pas
# # ## demander à Eric de relancer la machine sur sandbox
# # # get_call_info()

# # # get_call_info_europa()

# # proj = projects_load()
# # proj_id_signed = proj.project_id.unique()

# # prop = proposals_load()
# # proj = proj_add_cols(prop, proj)

# # stage_p =  ['REJECTED' ,'NO_MONEY' ,'MAIN', 'RESERVE', 'INELIGIBLE', 'WITHDRAWN', 'INADMISSIBLE', None]
# # prop1 = proposals_status(prop, proj_id_signed, stage_p)  
# # # np.save("data_files/applicants_columns.npy", prop_cols)

# # ###########################################
# # # proposals fix
# # # projects missing from proposals
# # call_to_integrate, call_miss = proposals_id_missing(prop1, proj, extractDate)

# # # project data missing in proposals if call already in proposals then add this
# # proj1 = proj_id_miss_fixed(prop1, proj, call_to_integrate)
# # call_miss = list(set(call_miss)-set(call_to_integrate))
# # proj = proj.loc[~proj.callId.isin(call_miss)]

# # # merge proj + prop
# # print('### MERGED PROPOSALS/PROJECTS')
# # if len(proj1)==0:
# #     prop2=pd.concat([proj,prop1], ignore_index= True)
# # else:
# #     prop2 = pd.concat([prop1, proj1, proj], ignore_index = True)

# # prop2 = prop2.loc[~((prop2.status_code=='REJECTED')&(prop2.stage=='successful'))]
# # print(f"- result - merged all: {len(prop2)},\n{prop2[['stage','status_code']].value_counts()}")

# # merged = copy.deepcopy(prop2)
# # merged = dates_year(merged)
# # merged = strings_v(merged)
# # merged = url_to_clean(merged)
# # merged.mask(merged=='', inplace=True)
# # merged = empty_str_to_none(merged)      
# # merged.rename(columns={
# #     'freekw':'free_keywords',
# #     'callDeadlineDate':'call_deadline', 
# #     'callId':'call_id', 
# #     'submissionDate':'submission_date',
# #     'startDate':'start_date',
# #     'endDate':'end_date', 
# #     'ecSignatureDate':'signature_date'}, inplace=True)

# # if any(merged.loc[merged.stage=='successful', 'project_id'].value_counts()[merged.loc[merged.stage=='successful', 'project_id'].value_counts()> 1]):
# #     print(merged.loc[merged.stage=='successful', 'project_id'].value_counts()[merged.loc[merged.stage=='successful', 'project_id'].value_counts()> 1])

# # merged = merged_panels(merged)
# # merged = merged_topics(merged)
# # merged = merged_actions(merged)

# # # calls list
# # calls = call(PATH_SOURCE+FRAMEWORK+'/')

# # print("\n### CALLS+MERGED")
# # if len(merged.loc[merged.call_id.isnull()])>0:
# #         print(f"1 - ATTENTION : manque des call_id: {merged.loc[merged.call_id.isnull(), 'project_id']}")
# # else:
# #     call_id = merged[['call_id', 'call_deadline']].drop_duplicates()
# #     print(f"2 - CALL_ID de merged -> nb call+deadline: {len(call_id)}, nb call unique: {call_id.call_id.nunique()} ")

# # calls = calls_to_check(calls, call_id)

# # projects = projects_complete_cleaned(merged, extractDate)

# # #############################################################
# # ##### PARTICIPATIONS
# # part = participants_load(proj)
# # # conserve uniquement les projets présents dans proposals et applicants
# # part = part.loc[part.project_id.isin(projects.project_id.unique())]
# # print(f"- size part hors proj manquant: {len(part)}")
# # part = part_role_type(part)
# # part = erc_role(part, projects)

# # #### APPLICANTS
# # app = applicants_load(prop)
# # # conserve uniquement les projets présents dans proposals et applicants
# # app1 = app.loc[app.project_id.isin(projects.project_id.unique())] 
# # print(f"- size app1 hors proj exclus: {len(app1)}")

# # app_missing_pid = projects.loc[(projects.stage=='evaluated')&(~projects.project_id.isin(app1.project_id.unique())), 'project_id'].unique()
# # tmp = part[part.project_id.isin(app_missing_pid)]
# # app1 = part_miss_app(tmp, app1)

# # #redressement accelerator
# # acc_folio = pd.read_csv(f"{PATH_SOURCE}{FRAMEWORK}/eic_fund_portfolio.csv", sep=';', dtype={'PROPOSAL_NBR':str})
# # print(f"size acc_folio: {len(acc_folio)}")
# # acc = (app1.loc[(app1.project_id.isin(acc_folio.PROPOSAL_NBR.unique()))&(app1.role=='Coordinator'),['project_id', 'role']]
# #        .merge(acc_folio[['PROPOSAL_NBR','GRANT_REQUESTED']], how='inner', left_on='project_id', right_on='PROPOSAL_NBR')
# #        .drop(columns='PROPOSAL_NBR'))
# # print(f"size acc: {len(acc)}")
# # app1 = app1.merge(acc, how='left', on=['project_id', 'role'])
# # app1.loc[app1.requestedGrant.isnull(), 'requestedGrant'] = app1.GRANT_REQUESTED
# # app1.drop(columns=['GRANT_REQUESTED'], inplace=True)

# # app1 = app_role_type(app1)
# # app1 = erc_role(app1, projects)

# # del app

# # ####
# # # verification Etat des participations
# # part = check_multiP_by_proj(part)
# # app1 = check_multiA_by_proj(app1)


# # ### STEP2
# # lien = merged_partApp(app1, part)
# # lien = nuts_lien(app1, part, lien)

# # # ENTITIES
# # entities = entities_load(lien)

# # list_codeCountry = list(set(entities.countryCode.to_list()+lien.countryCode.to_list()))
# # countries, countryCode_err = country_load(FRAMEWORK, list_codeCountry)
# # if countryCode_err:
# #     ccode=json.load(open("data_files/countryCode_match.json"))
# #     for i in ccode:
# #         for k,v in i.items():
# #             lien.loc[lien.countryCode==k, 'countryCode'] = v
# #             entities.loc[entities.countryCode==k, 'countryCode'] = v
# # cc_code = (countries[['countryCode', 'country_code_mapping']]
# #            .rename(columns={'countryCode':'iso2', 'country_code_mapping':'iso3'})
# #            .drop_duplicates())

# # lien = lien.merge(cc_code, how='left', left_on='countryCode', right_on='iso2').drop(columns='iso2')
# # lien = lien.merge(cc_code, how='left', left_on='proposal_countryCode', right_on='iso2').drop(columns='iso2')
# # lien = (lien.rename(columns={'iso3_x':'country_code_mapping', 'iso3_y':'proposal_country_code_mapping'})
# #         .drop(columns=['countryCode', 'proposal_countryCode']))
# # lien.to_pickle(f"{PATH_CLEAN}lien.pkl")

# # entities = (entities.merge(cc_code, how='left', left_on='countryCode', right_on='iso2').drop(columns='iso2')
# #             .rename(columns={'iso3':'country_code_mapping'}))

# # part = part.merge(cc_code, how='left', left_on='countryCode', right_on='iso2').drop(columns='iso2').rename(columns={'iso3':'country_code_mapping'})
# # app1 = app1.merge(cc_code, how='left', left_on='countryCode', right_on='iso2').drop(columns='iso2').rename(columns={'iso3':'country_code_mapping'})

# # #ENTITIES +LIEN
# # # entities = entities_cleaning(entities)
# # entities.to_pickle(f"{PATH_SOURCE}entities.pkl")
# # entities_single = entities_single_create(entities, lien)
# # entities_info = entities_info_create(entities_single, lien)

# # ### step3

# # # ##################################
# # # # nouvelle actualisation ; à executer UNE FOIS
# # # ref_source = ref_source_load('ref')
# # # result, check_id_liste, identification = first_update(ref_source, entities_info, countries)

# # # # vérifier dans excel les nouveaux ID PATH_WORK/_check_id_result.xlsx
# # # IDchecking_results(result, check_id_liste, identification)

# # # id_verified = ID_resultChecked()
# # # new_ref_source(id_verified, ref_source, extractDate, part, app1, entities_single, countries)

# # # ########################################################################################################

# # # chargement du nouveau ref_source
# # ref_source = ref_source_load('ref')
# # ref, genPic_to_new = ref_source_2d_select(ref_source, 'HE')
# # entities_tmp = entities_tmp_create(entities_info, countries, ref)
# # print(f"size entities_tmp: {len(entities_tmp)}")
# # entities_tmp = entities_for_merge(entities_tmp)

# # ### Executer uniquement si besoin
# # # lid_source, unknow_list = ID_entities_list(ref_source)
# # # ror = ror_getRefInfo(lid_source)
# # # siren_siret = get_siret_siege(lid_source)
# # # paysage_id = ID_to_IDpaysage(lid_source, siren_siret)
# # # paysage, paysage_mires = paysage_getRefInfo(lid_source, siren_siret, paysage_old=None)
# # # paysage_category = IDpaysage_category(paysage)
# # # sirene = get_sirene(lid_source, sirene_old=None)

# # #############################################################################################################


# # ### merge entities_tmp + referentiel
# # # ROR
# # ### si besoin de charger ror pickle
# # ror = pd.read_pickle(f"{PATH_REF}ror_df.pkl")
# # entities_tmp = merge_ror(entities_tmp, ror, countries)

# # # PAYSAGE
# # ### si besoin de charger paysage pickle
# # paysage = pd.read_pickle(f"{PATH_REF}paysage_df.pkl")
# # # paysage_category = IDpaysage_category(paysage)
# # paysage_category = pd.read_pickle(f"{PATH_SOURCE}paysage_category.pkl")
# # cat_filter = category_paysage(paysage_category)
# # entities_tmp = merge_paysage(entities_tmp, paysage, cat_filter)

# # # SIRENE
# # ### si besoin de charger paysage pickle
# # sirene = pd.read_pickle(f"{PATH_REF}sirene_df.pkl")
# # entities_tmp = merge_sirene(entities_tmp, sirene)

# # entities_tmp.loc[(~entities_tmp.id.isnull())&(entities_tmp.entities_id.isnull()), 'entities_id'] = entities_tmp.id

# # if any(entities_tmp.siren.str.contains(';', na=False)):
# #     print("ATTENTION faire code pour traiter deux siren différents -> ce qui serait bizarre qu'il y ait 2 siren")

# # # IDENT with '-' : traitement des identifiants avec '-' pour regrouper multi-pic non identifiés
# # entities_tmp = IDpic(entities_tmp)
# # entities_tmp = entities_tmp.merge(get_source_ID(entities_tmp, 'entities_id'), how='left', on='entities_id')

# # ### groupe entreprises
# # # groupe = groupe_treatment('groupe_prov', 'groupe')
# # ### si besoin de charger groupe 
# # groupe = pd.read_pickle(f"{PATH_REF}groupe.pkl")
# # print(f"taille de entities_tmp avant groupe:{len(entities_tmp)}")
# # entities_tmp = merge_groupe(entities_tmp, groupe)


# # entities_tmp = entities_clean(entities_tmp)
# # entities_check_null(entities_tmp)

# # # traitement catégorie
# # entities_tmp = category_woven(entities_tmp, sirene)
# # entities_tmp = category_agreg(entities_tmp)
# # entities_info = entities_info_add(entities_tmp, entities_info, countries)
# # entities_info = cordis_type(entities_info)

# # entities_info = fix_countries(entities_info, countries)
# # entities_info = mires(entities_info)

# # del ref_source

# # #check entities with pic_id
# # print("### check enties fr avec id commençant par pic")
# # pd.set_option("display.max_rows", None, "display.max_columns", None)
# # print(entities_info[(entities_info.country_code=='FRA')&(entities_info.entities_id.str.contains('pic'))][['entities_id', 'entities_name']])

# # file_name = f"{PATH_CLEAN}entities_info_current2.pkl"
# # with open(file_name, 'wb') as file:
# #     pd.to_pickle(entities_info, file)

# # STEP4 - INDICATEURS

# part_step = entities_with_lien(entities_info, lien, genPic_to_new)
# proj_no_coord = proj_no_coord(projects)



# def applicants_calcul(part_step, app1):
#     '''Traitement des subventions proposals -> création calculated_proposal_subv'''
#     print("\n### CALCULS applicants")
#     subv_p = (part_step.loc[part_step.inProposal==True, ['project_id', 'generalPic', 'country_code_mapping', 'proposal_orderNumber', 'projNlien']]
#             .drop_duplicates()
#             .merge(app1[['project_id', 'generalPic', 'country_code_mapping', 'orderNumber', 'role', 'partnerType', 'erc_role', 'requestedGrant']], 
#                 how='inner',
#                 left_on=['project_id', 'generalPic', 'country_code_mapping', 'proposal_orderNumber'],
#                 right_on=['project_id', 'generalPic', 'country_code_mapping', 'orderNumber']))

#     print(f"0 - {'{:,.1f}'.format(app1['requestedGrant'].sum())}, {'{:,.1f}'.format(subv_p['requestedGrant'].sum())}")

#     subv_p['calculated_fund'] = np.where(subv_p['projNlien']>1, subv_p['requestedGrant']/subv_p['projNlien'], subv_p['requestedGrant'])
#     subv_p.drop(['projNlien', 'orderNumber'], axis=1, inplace=True)

#     if len(subv_p)!=len(app1):
#         print(f"1- ATTENTION ! {len(subv_p)-len(app1)} participations perdues entre app1 et subv_p")

#     app_sum = '{:,.1f}'.format(app1['requestedGrant'].sum())
    
#     if '{:,.1f}'.format(subv_p['requestedGrant'].sum()) == app_sum:
#         print("2- requests grants = subventions proposals OK")
#     else:
#         print(f"3- ATTENTION ! Ecart subventions proposals -> subv_orig:{app_sum}, après fusion:{'{:,.1f}'.format(subv_p['requestedGrant'].sum())}")
        
#     part_prop = (part_step.merge(subv_p, how='inner')[
#                 ['project_id',  'generalPic', 'proposal_orderNumber', 'proposal_participant_pic', 'participation_linked', 
#                 'erc_role', 'cordis_is_sme',  'flag_entreprise', 'groupe_id', 'groupe_name', 'groupe_acronym',
#                 'cordis_type_entity_code', 'cordis_type_entity_name_fr', 'cordis_type_entity_name_en', 'cordis_type_entity_acro', 
#                 'participation_nuts', 'region_1_name', 'region_2_name', 'regional_unit_name',
#                 'country_code', 'country_code_mapping', 'extra_joint_organization', 'role', 'partnerType', 'calculated_fund']]
#                 .rename(columns={'proposal_orderNumber':'orderNumber', 'proposal_participant_pic':'pic'})
#                 .assign(stage='evaluated'))

#     if '{:,.1f}'.format(part_prop['calculated_fund'].sum())==app_sum:
#         print("4- Etape part_prop/subv_p -> calculated_proposal_subv OK")
#     else:
#         print(f"5- ATTENTION ! bien vérifier le volume de calculated_proposal_subv dans PARTICIPATION FINALE :{'{:,.1f}'.format(part_prop['calculated_fund'].sum())}, subv_orig:{app_sum}")

#     print(f"- size part_prop: {len(part_prop)}")
#     return part_prop



# part_prop = applicants_calcul(part_step, app1)
# part_proj = participants_calcul(part_step, part)
# participation = participations_complete(part_prop, part_proj, proj_no_coord)

In [ ]:
# def entities_single_create(df, lien):

#     print("\n### ENTITIES SINGLE")
#     entities_single=df.groupby(['generalPic', 'country_code_mapping']).head(1)
#     print(f"- size entities after one selection pic+cc: {len(entities_single)}")

#     print(f"\n- {entities_single.generalState.value_counts()}")
#     if (entities_single.generalPic.nunique())==(lien.generalPic.nunique()):
#         print(f"\n1 - nombre de pics OK")
#     #si pas le m^me nombre de pics entre lien et entities
#     elif len(set(lien.generalPic.unique()))>len(set(df.generalPic.unique())):
#         pic_lien=list(set(lien.generalPic.unique()) - set(df.generalPic.unique()))
#         print(f"\n2 - pic_lien absent de entities_single {pic_lien}; faire code")

#     tmp=entities_single.groupby(['generalPic', 'country_code_mapping']).filter(lambda x: x['generalPic'].count() > 1.)
#     if not tmp.empty:
#         print(f"1 - ATTENTION doublon generalPic revoir code ci-dessous si besoin")
           
#     print(f"- size entities_single:{len(entities_single)}")
#     return entities_single


# entities_single = entities_single_create(entities, lien)
entities_single = entities_single_create(entities, lien)

def entities_info_create(entities_single, lien):
    print("\n### ENTITIES INFO")
    entities_info = (entities_single
                     .drop(['pic', 'cedex', 'countryCode_y','lastUpdateDate'], axis=1)
                     .drop_duplicates())

    if len(entities_info[['generalPic', 'country_code_mapping']].drop_duplicates())!=len(lien[['generalPic', 'country_code_mapping']].drop_duplicates()):
        print(f"1- ATTENTION ! size genPic+cc -> entities_info : \n{len(entities_info[['generalPic', 'country_code_mapping']].drop_duplicates())},  lien:{len(lien[['generalPic', 'country_code_mapping']].drop_duplicates())}")
    else:
        pass
    print(f"- size entities_info: {len(entities_info)}")
    return entities_info
entities_info = entities_info_create(entities_single, lien)
entities_info.columns

In [2]:
import pandas as pd
df=pd.DataFrame({'a':[1,2,2,3,4,4,5], 'b':[2,2,2,3,4,5,5]}, index=[0,1,1,2,3,5,5])
df.reset_index().drop_duplicates(subset=['index','b', 'a']).set_index('index')

,a,b
index,,
0,1,2
1,2,2
2,3,3
3,4,4
5,4,5
5,5,5


In [ ]:
# ref_source = ref_source_load('ref')
# ref, genPic_to_new = ref_source_2d_select(ref_source, 'HE')

def entities_tmp_create(entities_info, countries, ref):
    print("### create ENTITIES TMP pour ref")
    tab = entities_info.merge(countries[['country_code_mapping', 'country_name_mapping', 'countryCode_parent']], how='left', on='country_code_mapping')
    tmp = tab.merge(ref, how='inner', on=['generalPic','country_code_mapping'])
    print(f"- size entities_info before:{len(entities_info)}, size entities_info+ref -> tmp:{len(tmp)}, Pic unique tmp:{len(tmp.generalPic.unique())}")
    # entities only into entities_info
    print("# missing entities into ref")
    tmp2 = tab.merge(tmp[['generalPic','country_code_mapping']], how='left', on=['generalPic','country_code_mapping'], indicator=True).query('_merge=="left_only"').drop(columns=['_merge'])
    print(f"- entities_info en + -> (tmp2): {len(tmp2)}")
    if not tmp2.empty:
        # test lien avec ref voire si un identifiant seulement sur le generalPic
        tmp2 = tmp2.merge(ref.drop(columns='country_code_mapping').drop_duplicates(), how='inner', on='generalPic')
        print(f"- size lien tmp2 with ref: {len(tmp2)}")
        ## add tmp2 to tmp
        tmp1 = pd.concat([tmp, tmp2], ignore_index=True)
        # entities_info without id
        tmp1 = tab.merge(tmp1[['generalPic','country_code_mapping']], how='left',on=['generalPic','country_code_mapping'], indicator=True).query('_merge=="left_only"').drop(columns=['_merge'])
        print(f"- size entities_info without id -> tmp1: {len(tmp1)}")
        tmp = pd.concat([tmp1, tmp], ignore_index=True)

    if (len(tmp))!=(len(entities_info)):
        print(f"1 - ATTENTION!!! size result {len(tmp)} diff size entities_info {len(entities_info)}")
    print(f"- End size entities_tmp {len(tmp)}")
    return tmp
entities_tmp = entities_tmp_create(entities_info, countries, ref)
print(f"size entities_tmp: {len(entities_tmp)}")
entities_tmp = entities_for_merge(entities_tmp)

In [ ]:
ror = pd.read_pickle(f"{PATH_REF}ror_df.pkl")
def merge_ror(entities_tmp, ror, countries):
    print("### merge ROR")
    ccode=json.load(open("data_files/countryCode_match.json"))
    for i in ccode:
        for k,v in i.items():
            ror.loc[ror.country_code==k, 'country_code'] = v
            ror.loc[ror.country_code==k, 'country_code'] = v
    ror = (ror
           .merge(countries[['countryCode', 'country_code_mapping']], 
                  how='left', left_on='country_code', right_on='countryCode')
            .drop(columns=['countryCode', 'country_code']))

    entities_tmp = (entities_tmp
                    .merge(ror.drop(columns='country_code_mapping'), 
                           how='left', left_on=['id_extend'], right_on=['id_source'])
                    .drop(columns='id_source')
                    .drop_duplicates())
    print(f"- End size entities_tmp+ror_info: {len(entities_tmp)}")
    if any(entities_tmp.groupby('generalPic')['generalPic'].transform('count')>1):
        entities_tmp[entities_tmp.groupby('generalPic')['generalPic'].transform('count')>1]
    return entities_tmp
ror
entities_tmp = merge_ror(entities_tmp, ror, countries)

In [ ]:
paysage = pd.read_pickle(f"{PATH_REF}paysage_df.pkl")
# paysage_category = IDpaysage_category(paysage)
paysage_category = pd.read_pickle(f"{PATH_SOURCE}paysage_category.pkl")
cat_filter = category_paysage(paysage_category)
entities_tmp = merge_paysage(entities_tmp, paysage, cat_filter)
sirene = pd.read_pickle(f"{PATH_REF}sirene_df.pkl")
entities_tmp = merge_sirene(entities_tmp, sirene)

entities_tmp.loc[(~entities_tmp.id.isnull())&(entities_tmp.entities_id.isnull()), 'entities_id'] = entities_tmp.id

if any(entities_tmp.siren.str.contains(';', na=False)):
    print("ATTENTION faire code pour traiter deux siren différents -> ce qui serait bizarre qu'il y ait 2 siren")


In [ ]:
entities_tmp = IDpic(entities_tmp)
entities_tmp = entities_tmp.merge(get_source_ID(entities_tmp, 'entities_id'), how='left', on='entities_id')

### groupe entreprises
# groupe = groupe_treatment('groupe_prov', 'groupe')
### si besoin de charger groupe 
groupe = pd.read_pickle(f"{PATH_REF}groupe.pkl")
print(f"taille de entities_tmp avant groupe:{len(entities_tmp)}")
entities_tmp = merge_groupe(entities_tmp, groupe)


In [ ]:
entities_tmp = entities_clean(entities_tmp)
entities_check_null(entities_tmp)

# traitement catégorie
entities_tmp = category_woven(entities_tmp, sirene)
entities_tmp = category_agreg(entities_tmp)
def entities_info_add(entities_tmp, entities_info, countries):
    print("\n### entities_info + entities_tmp")
        #ajout des infos country à participants_info
    entities_info = (entities_info
                    .merge(countries[['country_code_mapping', 'country_name_mapping', 'country_code']], how='left', on='country_code_mapping'))
        
    entities_info = (entities_info
        .merge(entities_tmp[
            ['generalPic', 'id', 'ZONAGE', 'id_m', 'siren', 'country_code_mapping',
            'id_secondaire', 'entities_id',  'entities_name', 'entities_acronym', 
            'insee_cat_code', 'insee_cat_name',  'category_agregation',
            'paysage_category', 'flag_entreprise', 
            'ror_category', 'category_woven', 'source_id', 'sector',  
            'siret_closeDate', 'cat_an',
            'groupe_name','groupe_acronym', 'groupe_id', 'groupe_sector']],
        how='left', on=['generalPic', 'country_code_mapping'])
        .drop(columns=['legalName', 'businessName']))
    print(f"- size entities_info + entities_tmp: {len(entities_info)}")
    return entities_info
entities_info = entities_info_add(entities_tmp, entities_info, countries)
entities_info = cordis_type(entities_info)

entities_info = fix_countries(entities_info, countries)
entities_info = mires(entities_info)